# Caffe的Python接口学习(5)：生成deploy文件

如果要把训练好的模型拿来测试新的图片，那必须得有一个deploy.prototxt文件，这个文件实际上和test.prototxt文件差不多，只是头尾不相同而已。deploy文件没有第一层数据输入层，也没有最后的Accuracy层，但最后多了一个Softmax概率层。

这里我们采用代码的方式来自动生成该文件，以mnist为例。

deploy.py：
``` python
# -*- coding: utf-8 -*-

from caffe import layers as L, params as P, to_proto

root = '/root/Repository/caffe/examples/mine/'
deploy = root + 'mnist/deploy.prototxt' # 文件保存路径

def create_deploy():
    # 少了第一层：data层
    conv1 = L.Convolution(bottom='data', kernel_size=5, stride=1, num_output=20, pad=0, 
                          weight_filler=dict(type='xavier'))
    pool1 = L.Pooling(conv1, pool=P.Pooling.MAX, kernel_size=2, stride=2)
    conv2 = L.Convolution(pool1, kernel_size=5, stride=1, num_output=50, pad=0, 
                          weight_filler=dict(type='xavier'))
    pool2 = L.Pooling(conv2, pool=P.Pooling.MAX, kernel_size=2, stride=2)
    fc3 = L.InnerProduct(pool2, num_output=500, weight_filler=dict(type='xavier'))
    relu3 = L.ReLU(fc3, in_place=True)
    fc4 = L.InnerProduct(relu3, num_output=10, weight_filler=dict(type='xavier'))
    # 没有最后的accuracy层，但有一个Softmax层
    prob = L.Softmax(fc4)
    return to_proto(prob)

def write_deploy():
    with open(deploy, 'w') as f:
        f.write('name: "LeNet"\n')
        f.write('input: "data"\n')
        f.write('input_dim: 1\n')
        f.write('input_dim: 3\n')
        f.write('input_dim: 28\n')
        f.write('input_dim: 28\n')
        f.write(str(create_deploy()))
        
if __name__ == '__main__':
    write_deploy()
```

运行该文件后，会在mnist目录下，生成一个deploy.prototxt文件。

这个文件不推荐使用代码来生成，反而麻烦。大家熟悉以后可以将test.prototxt复制一份，修改相应的地方就可以了，更加方便。